In [1]:
%load_ext watermark
%watermark

ModuleNotFoundError: No module named 'watermark'

In [2]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [24]:
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

import tkinter as tk
from tkinter import filedialog

import os

from scipy import signal
import scipy.integrate as integrate

# %matplotlib qt

In [659]:
def lin_func(x,a,b):
    return a*x+b
def exp_sigmoid(x,a,b,c,d):
    return a/(1+exp(-c*(x+d))) + b
def slexp_sigmoid(x,a,b,c,d):
    return sqrt(-log(exp_sigmoid(-x,1,0,1,0)))
def slexp_sigmoid2(x,a,c,t0,e):
    return e*sqrt(-log( a/(1+exp(c*x-t0)) ))
def logistic(x,K,a,r):
    return K/(1+a*exp(-r*x))
def logistic2(x,a,beta,xc,maxL,minL):
    return (maxL-minL)/(1+a*exp(beta*(-x/xc)))+minL
def five_param_logistic(x,p1,p2,p3,p4,p5):
    c = 2*p3*p5/abs(p3+p5)
    f = 1/(1+exp(-c*(p4-x)))
    g = exp(p3*(p4-x))
    h = exp(p5*(p4-x))
    return p1 + p2/( 1+f*g+(1-f)*h )
def func2(t, t0, A, B,  C, D):
    return A*(t-t0) / (C+(t-t0)**2)**D + B

In [482]:
x1 = linspace(-6,6,100)
#y = exp_sigmoid(x,1,0,1,-1)
#y = slexp_sigmoid2(x,1,1000,1.7,1)
y1 = logistic(x,1,1,1)*200
x2 = linspace(-10,10,100)
y2 = logistic2(x,3,1,1,200,1e-2)
figure(0,clear='True')
plot(x1,y1,'P')
plot(x2,y2,'P')
grid()
tight_layout()

In [4]:
n_core = 8
N = 512
print(linspace(1,N+1-N//n_core,n_core,dtype='int'))
print(linspace(N//n_core,N,n_core,dtype='int'))

n_dt = 5000

# integer          , parameter :: ia(ni)=[1,257,513,769]
# integer          , parameter :: ib(ni)=[256,512,768,1024]

[  1  65 129 193 257 321 385 449]
[ 64 128 192 256 320 384 448 512]


In [5]:
%run functions
matplotlib.rcParams.update({'font.size': 25})

In [6]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol

wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [664]:
file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220617' # 20220610
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220512'
# file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/20220510'

all_subdir = []
all_Temp = []
all_trjN = []
all_trj1 = []
all_xva_3DHarmo = []
all_xva_start = []

# selection of all directories where
# there are data files
for i,j in enumerate(os.walk(file_path)):
    if 'Try' in j[0]:
        all_subdir.append(j[0])
    # selection of all data files
    # interesting me
    for k,l in enumerate(j[2]):
        if 'TrjN' in l:
            all_trjN.append(j[0]+'/'+l)
        if 'Trj1' in l:
            all_trj1.append(j[0]+'/'+l)
            #print(l)
        if 'SimuType0' and '.dat' in l:
            all_Temp.append(j[0]+'/'+l)
        if 'xva_3D' in l:
            all_xva_3DHarmo.append(j[0]+'/'+l)
        if 'xva_start' in l:
            all_xva_start.append(j[0]+'/'+l)
        #print(j[0],'\n')

# for i in range(len(all_subdir)):
#     print(all_subdir[i],'\n',all_Temp[i],'\n',all_trjN[i],'\n')

In [665]:
all_Temp,all_trj1

(['/home/adrien/RemoteFS/Rivendel/Simulations/20220617/DC07_RF07/Try00/Temp_SimuType0_N00512_Vrf0071_Udc0.2116D+02V_nt1000.dat',
  '/home/adrien/RemoteFS/Rivendel/Simulations/20220617/DC06_RF06/Try00/Temp_SimuType0_N00512_Vrf0061_Udc0.1424D+02V_nt1000.dat',
  '/home/adrien/RemoteFS/Rivendel/Simulations/20220617/DC04_RF04/Try00/Temp_SimuType0_N00512_Vrf0042_Udc0.6010D+01V_nt1000.dat',
  '/home/adrien/RemoteFS/Rivendel/Simulations/20220617/DC05_RF05/Try00/Temp_SimuType0_N00512_Vrf0051_Udc0.9330D+01V_nt1000.dat',
  '/home/adrien/RemoteFS/Rivendel/Simulations/20220617/DC03_RF03/Try00/Temp_SimuType0_N00512_Vrf0031_Udc0.3160D+01V_nt1000.dat'],
 [])

In [676]:
# OK pour la temperature
load_number = 0
print(all_Temp[load_number])
data = loadtxt(all_Temp[load_number])
time = data[:,0]
T_CM = data[:,1:4]
T_aux = data[:,4:]
T_aux_avg = mean(T_aux,axis=1)

# test plot
index_max = 1900
figure(1,clear='True')
semilogy(time*1e3,T_aux,'P')
semilogy(time*1e3,T_aux_avg,color='r')
vlines(time[index_max]*1e3,1e-3,100)
# vlines(time[2000]*1e3,1e-1,1)
xlabel('time [ms]')
ylabel('T [K]')
# xlim(1,1.1)
grid()
tight_layout()

/home/adrien/RemoteFS/Rivendel/Simulations/20220617/DC07_RF07/Try00/Temp_SimuType0_N00512_Vrf0071_Udc0.2116D+02V_nt1000.dat


In [518]:
# same as above
# with fit temperature

# OK pour la temperature
load_number = 0
print(all_Temp[load_number])
data = loadtxt(all_Temp[load_number])
time = data[:,0]
T_CM = data[:,1:4]
T_aux = data[:,4:]
T_aux_avg = mean(T_aux,axis=1)

x = linspace(1,1.9,1000)
# y = exp_sigmoid(x,1,0,1,-1) for log plot
# y = sqrt(-log(exp_sigmoid(-x,1,0,1,0))) # pas bon


# test plot
# logistic(x,K,a,r):
index_max = 1900
# p0 = array([50,1e-3,50])
popt, pcov = curve_fit(logistic, time[:index_max], log10(T_aux_avg[:index_max]),
                       p0,maxfev= 500000)
fit_param = popt[0], popt[1], popt[2]
fit_pcov  = pcov
print(fit_param)

figure(11,clear='True')
semilogy(time*1e3,T_aux_avg,color='r')
plot(time[:index_max]*1e3,logistic(time[:index_max]*1e3,50,1000,1),color='b')
plot(time[:index_max]*1e3,logistic(time[:index_max]*1e3,*fit_param),color='k')

xlabel('time [ms]')
ylabel('T [K]')
xlim(0.9,2.1)
grid()
tight_layout()



/home/adrien/RemoteFS/Rivendel/Simulations/20220610/DC07_RF07/Try00/Temp_SimuType0_N00512_Vrf0071_Udc0.2116D+02V_nt1000.dat


/usr/lib/python3/dist-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/tmp/ipykernel_8957/2691313375.py:39: RuntimeWarning: invalid value encountered in log
  logit_T_aux_avg = log(T_aux_avg/(1-T_aux_avg))


(0.12016035163163255, 0.13725077089097168, 119423.98527906199)


In [543]:
# fit par logistic
# en utilisant logit

T_aux_avg_log = log(T_aux_avg_norm)
T_aux_avg_lognorm = T_aux_avg_log + min(T_aux_avg_log)
T_aux_avg_lognorm = T_aux_avg/max(T_aux_avg)

logit_p = log(T_aux_avg_lognorm/(1-T_aux_avg_lognorm))
#T_aux_avg_log = log(T_aux_avg_norm)/max(log(T_aux_avg_norm))
#logit_T_aux_avg = log(T_aux_avg_log/(1-T_aux_avg_log))
figure(12,clear='True')
plot(time*1e3,logit_p/T_aux_avg_log,color='b')

xlabel('time [ms]')
ylabel('T [K]')
xlim(0.9,2.1)
grid()
tight_layout()

/tmp/ipykernel_8957/1270371780.py:8: RuntimeWarning: divide by zero encountered in true_divide
  logit_p = log(T_aux_avg_lognorm/(1-T_aux_avg_lognorm))


In [651]:
# test plot
index_max = 1900
# five_param_logistic(x,p1,p2,p3,p4,p5)
# p0 = array([50,1e-8,50,-1.6])
popt, pcov = curve_fit(five_param_logistic, time[:index_max]*1e3, log(T_aux_avg[:index_max]),
                        maxfev= 500000)
fit_param = popt[0], popt[1], popt[2], popt[3], popt[4]
fit_pcov  = pcov

figure(11,clear='True')
#semilogy(time*1e3,T_aux_avg,color='r')
#semilogy(time[:index_max]*1e3,five_param_logistic(time[:index_max]*1e3,*popt),color='b')
#plot(time*1e3,log(T_aux_avg),color='r')
plot((time)*1e3,
         five_param_logistic((time-min(time))*1e3,1e-2,50,1,1,300),
         color='k')
#plot(time[:index_max]*1e3,exp_sigmoid(time[:index_max]*1e3,*fit_param),color='k')

xlabel('time [ms]')
ylabel('T [K]')
#xlim(1,2.1)
#ylim(-1e-3,55)
grid()
tight_layout()

/tmp/ipykernel_8957/1191193704.py:15: RuntimeWarning: overflow encountered in exp
  f = 1/(1+exp(-c*(p4-x)))
/usr/lib/python3/dist-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [49]:
# plt.style.use('default')
%run functions

In [50]:
# Retrieve r² and v² data

print(all_trjN[load_number])
r2_v2 = load_xv2rlim(all_trjN[load_number])

/home/adrien/RemoteFS/Rivendel/Simulations/20220610/DC07_RF07/Try00/xva_SimuType0_N00512_Vrf0071_Udc0.2116D+02V_nt1000_TrjN.bin
[720000]
[720000]


In [123]:
# test plot r² and v²

figure(2,clear='True')
for k in range(3):
    plot(time*1e3,r2_v2[k,:len(time)],label=str(k))
xlabel('time [ms]')
ylabel('R² [m²]')
# ylim(0,100)
grid()

legend()
tight_layout()

figure(3,clear='True')
for k in range(3):
    semilogy(time*1e3,r2_v2[k+3,:len(time)],label=str(k))
xlabel('time [ms]')
ylabel('V² [m/s ²]')
grid()
legend()
tight_layout()

figure(21,clear='True')
for k in range(3):
    plot(time*1e3,r2_v2[k+6,:len(time)],label=str(k))
xlabel('time [ms]')
ylabel('R_lim [m]')
grid()
tight_layout()

b, a = signal.butter(8, 0.125)
R_lim_x_filt = signal.filtfilt(b, a, r2_v2[6,:len(time)], padlen=150)
figure(22,clear='True')
plot(T_aux[:,0],r2_v2[6,:len(time)],label=str(k),marker='+',ls='')
a = 2.5e-3
b = 0.085
plot(linspace(0,40,1000),a*linspace(0,40,1000)+b,label=str(k),ls='-')
a = 2.7e-3
b = 0.075
plot(linspace(0,40,1000),a*linspace(0,40,1000)+b,label=str(k),ls='-')
xlabel('Temperature [K]')
ylabel('R_lim [m]')
xlim(-0.5,42)
ylim(0.02,0.22)
grid()
tight_layout()

In [ ]:
t = time - max(time)
T = max(t)-min(t)
dt = time[1]-time[0]
N_samp = len(time)
print(1/dt)
print( 'fmax =',1/(2*dt),'Hz' )
print( 'df =',1/(len(t)*dt),'Hz' )

y = r2_v2[5,:len(t)]
w = blackman(len(t))
ywf = fft(y*w)
xf = fftfreq(len(t), dt)[:N_samp//2]
figure(31,clear='True')
plt.semilogy(xf[1:N_samp//2], 2.0/N_samp * np.abs(ywf[1:N_samp//2]), '-r')
plt.tight_layout()

In [19]:
# q = 0.56
Urf = 42
Udc = 6.05
q = 4*C_e*Urf/(m_Ca*r0**2*Omega**2)
print('q',q)
q_fact = (1+1/8*q**2)
q_omega = 1/8*q**2*Omega**2

beta_guess = 0.423
a = 0
for k in range(15):
    beta_guess = beta_continue_alamano(a,q,beta_guess)
    beta_guess = sqrt(beta_guess)
#     print(beta_guess)
print('non adiab',beta_guess)
print('adiab    ',sqrt(a+q**2/2))
print('non adiab',0.5*beta_guess*Omega/1000/2/pi)
print('adiab    ',0.5*sqrt(a+q**2/2)*Omega/1000/2/pi)

omega_x = beta_guess*Omega/2 # 462249.463 *2*pi # 404799.82030008605
omega_y = omega_x
omega_z = 100000*sqrt(Udc) *2*pi

Ekin = 0.5*m_Ca*( q_fact*(r2_v2[3]+r2_v2[4]) + r2_v2[5] + q_omega*(r2_v2[0]+r2_v2[1]) )

Ekin_nig = 0.5*m_Ca * ( (q_fact*omega_x**2+q_omega)*r2_v2[0] \
                + (q_fact*omega_y**2+q_omega)*r2_v2[1] \
                + (omega_z**2)*r2_v2[2] )

Heating_rate = Ekin[:len(time)] / Ekin_nig[:len(time)]

q 0.39000388637797107
non adiab 0.28474130089623617
adiab     0.27577439274697113
non adiab 291.85983341864204
adiab     282.66875256564543


In [20]:
figure(4,clear='True')
semilogy(time*1e3,Heating_rate[:len(time)])
xlabel('time [ms]')
ylabel('h [J/J /period]')
grid()

#semilogy(time,Ekin[:len(time)])
#semilogy(time,Ekin_nig[:len(time)])
tight_layout()

In [ ]:
# plot dE/dt always same problem (centered on 0)
# plot dE/dT ? same pb
figure(5,clear='True')
plot(mean(T_aux,axis=1)[:-1],diff(Ekin[:len(time)]))
xlabel('temperature [T]')
ylabel('diff(Ekin)')
grid()
tight_layout()

In [21]:
# Blumel style
cloud_size   = sqrt(r2_v2[0]+r2_v2[1]+r2_v2[2])
crystal_size = sqrt(r2_v2[0,5]+r2_v2[1,5]+r2_v2[2,5])

figure(6,clear='True')
plot(cloud_size[1:len(time)]/crystal_size,Heating_rate[1:len(time)])
xlabel('size sigma')
ylabel('h [J/J /period]')
grid()
tight_layout()

In [ ]:
# Drewsen style
R = 88*1e-6
V = 4/3*pi*R**3
rho = N/V
aws = (1/rho)**(1/3)
temperature_gamma = C_e**2 / (4*pi*eps0*aws*kb*mean(T_aux,axis=1))

figure(61,clear='True')
plot(mean(T_aux,axis=1)[1:len(time)],Ekin[1:len(time)],
     marker='.',
     label='$E_{kin}^{adia}$') # Heating_rate Ekin[:len(time)]
plot(mean(T_aux,axis=1)[1:len(time)],Ekin_nig[1:len(time)],
     marker='.',
     label='$E_{kin}^{nig}$')
plot(mean(T_aux,axis=1)[1:len(time)],1/2*kb*N*n_dt**2*mean(T_aux,axis=1)[1:len(time)],
     marker='.',
     label='$E_{kin}^{nig}$')
xlabel('Temperature [K]')
ylabel('$E_{kin}^{adia}$ [J]')
grid()
tight_layout()

figure(65,clear='True')
plot(mean(T_aux,axis=1)[1:len(time)],
     Ekin[1:len(time)] / (1/2*kb*N*n_dt**2*mean(T_aux,axis=1)[1:len(time)]),
     marker='.',
     label='$E_{kin}^{nig}$')
xlabel('Temperature [K]')
ylabel('$E_{kin}^{adia}$ [J]')
grid()
tight_layout()

figure(66,clear='True')
plot(time*1e3,
     temperature_gamma,
     marker='.',
     label='$E_{kin}^{nig}$')
xlabel('time [ms]')
ylabel('$\Gamma$')
grid()
tight_layout()

# figure(62,clear='True')
# plot(1/temperature_gamma[1:len(time)],Ekin_nig[1:len(time)],'+') # Heating_rate Ekin[:len(time)]
# xlabel('Temperature [1/Gamma]')
# ylabel('$E_{kin}^{nig}$ [J]')
# grid()
# tight_layout()

# figure(64,clear='True')
# loglog(1/temperature_gamma[1:len(time)],Heating_rate[1:len(time)],'+') # Heating_rate Ekin[:len(time)]
# xlabel('Temperature [1/Gamma]')
# ylabel('H [J/s]')
# grid()
# tight_layout()
# Ekin[:len(time)] / Ekin_nig[:len(time)]

In [287]:
figure(7,clear='True')
b, a = signal.butter(8, 0.125)
cloud_size_filt = signal.filtfilt(b, a, cloud_size[1:len(time)], padlen=150)
plot(time[1:],cloud_size_filt*1e-6)
xlabel('time [s]')
ylabel('s [µm]')
xlim(0.001,0.0025)
ylim(0,4)

grid()
tight_layout()

In [ ]:
# Retrieve r at end langevin
load_number = 1
file_to_open = all_xva_3DHarmo[load_number] # [load_number]all_xva_start
print(file_to_open)
xva_3DHarmo = load_x_3DHarmo(file_to_open)

In [ ]:
# Retrieve r at end langevin
load_number = 1
file_to_open = all_trj1[load_number] # [load_number]all_xva_start
print(file_to_open)
xva_trj1 = load_x_afterstart(file_to_open)

In [ ]:
time_thin = linspace(0,time[-1]-min(time),len(xva_trj1[0]))
n_dt = 5000
offset = 0*n_dt
start = 0 + offset
end = 200*n_dt + offset

dim_to_plot = 0
figure(88,clear='True')
plot(time_thin[start:end]*1e6,xva_trj1[dim_to_plot,start:end]*1e6)
xlabel('t [µs]')
ylabel('x [µm]')
grid()
tight_layout()

T = max(time_thin[start:end])
dt = time_thin[1]-time_thin[0]
N_samp = len(time_thin[start:end])
# print(1/dt)
# print( 'fmax =',1/(2*dt),'Hz' )
# print( 'df =',1/(len(t)*dt),'Hz' )
y = xva_trj1[dim_to_plot,start:end]
w = blackman(len(y))
len_pad = 2**20
ywf = fft(y*w,n=len_pad)
xf = fftfreq(len(y), dt)[:N_samp//2]
figure(89,clear='True')
plt.semilogy(xf[1:N_samp//2], 2.0/N_samp * np.abs(ywf[1:N_samp//2]), '-+r')
xlim(0,1e7)
plt.tight_layout()

In [ ]:
figure(8,clear='True')
scatter(xva_3DHarmo[0]*1e6,xva_3DHarmo[1]*1e6)
xlabel('x [µm]')
ylabel('y [µm]')
grid()
tight_layout()

figure(9,clear='True')
scatter(xva_3DHarmo[2]*1e6,xva_3DHarmo[0]*1e6)
xlabel('z [µm]')
ylabel('x [µm]')
grid()
tight_layout()

figure(10,clear='True')
scatter(xva_3DHarmo[2]*1e6,xva_3DHarmo[1]*1e6)
xlabel('z [µm]')
ylabel('y [µm]')
grid()
tight_layout()

In [ ]:
# Compare Ekin pseudopot to ekin measured

figure(11,clear='True')
plot(time*1e3,0.5*m_Ca*sum(T_aux,axis=1)/m_Ca*kb*N*1000**2,label='Ekin meas')
plot(time*1e3,Ekin[:len(time)],'--',label='Ekin pseudopot')
xlabel('time [ms]')
ylabel('Energy [J]')
grid()
legend()
tight_layout()

figure(12,clear='True')
plot(time*1e3,Ekin[:len(time)] - 0.5*m_Ca*sum(T_aux,axis=1)/m_Ca*kb*N*1000**2)
xlabel('time [ms]')
ylabel('Energy diff [J]')
grid()
tight_layout()

figure(num=122,clear='True')
plot(time*1e3,Ekin[:len(time)]/(0.5*m_Ca*sum(T_aux,axis=1)/m_Ca*kb*N*1000**2))
xlabel('time [ms]')
ylabel('Energy ratio [J]')
grid()
tight_layout()

# The problem here is that the energy measured in the simu
# is significantly different from the energy in pseudo-pot approx
# also Blümel relies on average values at equilibrium
# so his h=E/E do not work for me

In [ ]:
# trim data from dirty edges
start = 5
end = len(time)-50
time_good = time[start:end]
T_good = mean(T_aux,axis=1)[start:end]
cloud_size   = sqrt(r2_v2[0,start:end]+r2_v2[1,start:end]+r2_v2[2,start:end])
crystal_size = sqrt(r2_v2[0,5]+r2_v2[1,5]+r2_v2[2,5])
# filter data
b, a = signal.butter(8, 0.125)
y = signal.filtfilt(b, a, T_good, padlen=150)

In [ ]:
figure(14,clear='True')
plot(time_good*1e3,T_good)
plot(time_good*1e3,y,'--P')
vlines(time_good[1500]*1e3,0,50)
vlines(time_good[2000]*1e3,0,50)
grid()
#xlim(35.3,37.4)
#ylim(-1,75)
xlabel('time [ms]')
ylabel('Temperature [K]')
tight_layout()

In [ ]:
# Heating by differentiating
# between points spaced by 50 RF periods
time_step = 488 # 488 # 488
N_period = (time_good[time_step]-time_good[0])*(Omega/2/pi)
y_diff = (y[time_step:]-y[:-time_step])
time_good_diff = time_good[time_step//2:-time_step//2]*1e3

figure(141,clear='True')
plot(time_good[:-time_step]*1e3,
     y_diff/N_period)
grid()
#xlim(35.3,37.4)
#ylim(-1,75)
xlabel('time [ms]')
ylabel('Heating rate [K/period]')
tight_layout()

figure(142,clear='True')
plot(y[:-time_step],
     y_diff/N_period)
grid()
#xlim(35.3,37.4)
#ylim(-1,75)
xlabel('Temperature [K]')
ylabel('Heating rate [K/period]')
tight_layout()

figure(1421,clear='True')
plot(y[1:-time_step],
     diff(y_diff/N_period))
grid()
#xlim(35.3,37.4)
#ylim(-1,75)
xlabel('Temperature [K]')
ylabel('Heating rate [K/period]')
tight_layout()

R = 88*1e-6
V = 4/3*pi*R**3
rho = N/V
aws = (1/rho)**(1/3)
temperature_gamma = C_e**2 / (4*pi*eps0*aws*kb*y)

figure(143,clear='True')
plot(1/temperature_gamma[:-time_step],
     y_diff/N_period)
grid()
#xlim(35.3,37.4)
#ylim(-1,75)
xlabel('Temperature [1/$\Gamma$]')
ylabel('Heating rate [K/period]')
tight_layout()

figure(144,clear='True')
loglog(1/temperature_gamma[:-time_step],
     (Ekin[:len(y_diff)]/(y_diff/N_period))**-1)
grid()
#xlim(35.3,37.4)
#ylim(-1,75)
xlabel('Temperature [1/$\Gamma$]')
ylabel('Heating rate [K/period]')
tight_layout()

In [ ]:
print(N_period)

In [ ]:
figure(15,clear='True')
plot(y[:-1],diff(y),'--P')
xlabel('T [K]')
ylabel('H [K/S]')
grid()
# xlim(35.3,37.4)
# ylim(-0.05,0.15)
tight_layout()

In [ ]:
figure(16,clear='True')
plot(y[:-1],diff(y)/y[:-1],'--P')
xlabel('T [K]')
ylabel('H/T [K/S /K]')
grid()
# xlim(35.3,37.4)
# ylim(-0.05,0.15)
tight_layout()

In [ ]:
len(y)

In [ ]:
figure(17,clear='True')
plot(cloud_size[:-1]/crystal_size,diff(y),'--P')
xlabel('size [m]')
ylabel('H [K/S]')
grid()
# xlim(35.3,37.4)
# ylim(-0.05,0.15)
tight_layout()

figure(18,clear='True')
plot(cloud_size[:-1]/crystal_size,diff(y)/y[:-1],'--P')
xlabel('size [m]')
ylabel('H/T [K/S /K]')
grid()
# xlim(35.3,37.4)
# ylim(-0.05,0.15)
tight_layout()

# Extract data as npz

In [ ]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint')
print(file_path)

In [ ]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

In [ ]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

In [ ]:
## Data loading (1D arrays)
data_name, num_runs, Time, Temperature \
= data_retrieve(all_subdir,points_and_coord,condition_parameters,slash_cfg) # stopping_power  ,forcenumtry=28

In [ ]:
for j,k in enumerate(data_name[0]):
    cond = k[0][52:61]
    print(cond)
    outfile = 'Time_and_temp_RFHEAT_N0512_'+str(cond)
    np.savez( outfile, time=Time[j], temp = mean(Temperature[j],axis=0)) # dirname_hc=re.sub(r'\\',r'/',dirname_hc)

In [ ]:
# load check
cond = 'DC11_RF18'
outfile = 'Time_and_temp_RFHEAT_N0512_'+str(cond)
with load(outfile+'.npz') as data:
    a = data['time']
    b = data['temp']
print(a)
print(b)
print(shape(a))
print(shape(b))

In [ ]:
whos

# Work with data

In [ ]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python')
print(file_path)

for j,k in enumerate(file_path):
    cond = k[0][52:61]

In [ ]:
all_Urf = [60,61,62,63,64,65,66,67,68,62.2,62.4,62.6,62.8,
       63.2,63.4,63.6,63.8,40,45,50,55,70,75,80,85]
Urf = []
for j,k in enumerate(file_path):
    str_pos = k.find('N0512_')
    cond = k[str_pos+6:str_pos+15]
    str_pos = cond.find('RF')
    RF_cond = cond[str_pos+2:str_pos+4]
    print(cond,RF_cond)
    Urf.append(all_Urf[int(RF_cond)])
print(sort(Urf))

In [ ]:
# load check
# cond = 'DC11_RF00'
# outfile = 'Time_and_temp_RFHEAT_'+str(cond)
Time = []
Temperature = []
for j,k in enumerate(file_path):
    with load(k) as data:
        Time.append( data['time'] )
        Temperature.append( data['temp'] )
    print(j,k)
print(shape(Time))
print(shape(Temperature))

In [ ]:
imax = len(Urf)
print('imax',imax)

T_max = 50
T_thr = 1

figure(10); clf();
i = 11
print(i,Urf[i])
#     subplot(1,9,i+1)
semilogy(Time[i]*1e3,savgol_filter(Temperature[i],31,1),label=' Vrf = '+str(Urf[i]))
#     semilogy(time[i][:]*1e3,Temp[i][:],label=' Vrf = '+str(Urf[i]))
#         semilogy(time[k_thr],Temp[i,j,k_thr],'o')
grid()
legend(fontsize=12)

In [ ]:
# plot all temperature curves

imax = len(Urf)
print('imax',imax)

T_max = 50
T_thr = 1

figure(1); clf();
for i in range(0,imax-1):
    print(i)
    k_max = argmin(abs(Temperature[i] - T_max))
    k_thr = argmin(abs(Temperature[i] - T_thr))
    k_min = k_thr - (k_max - k_thr)
    if k_min < 0:
        k_min = 0

    i_aux = where(diff(abs(diff(Temperature[i][k_thr:k_max])))>2)
    if size(i_aux) != 0 :
        if (i_aux[0][0]+k_thr < k_max):
            k_max = i_aux[0][0]+k_thr

#     subplot(1,9,i+1)
    subplot(imax//3,4,i+1)
    semilogy(Time[i][k_min:k_max]*1e3,Temperature[i][k_min:k_max],label=' Vrf = '+str(Urf[i]))
#     semilogy(time[i][:]*1e3,Temp[i][:],label=' Vrf = '+str(Urf[i]))
#         semilogy(time[k_thr],Temp[i,j,k_thr],'o')
    grid()
    legend(fontsize=12)
tight_layout()

In [ ]:
# Functions for fit

def func2(t, t0, A, B,  C, D):
    return A*(t-t0) / (C+(t-t0)**2)**D + B

def func3(t, t0, A, B,  C, D):
    return A*(t-t0) / (C+abs(t-t0))**D + B

def dfunc2_dt(t, t0, A, B,  C, D):
    return 10**(A*(C + (t - t0)**2)**(-D)*(t - t0) + B)*(-A*D*(C + (t - t0)**2)**(-D)*(t - t0)*(2*t - 2*t0)/(C + (t - t0)**2) + A*(C + (t - t0)**2)**(-D))*log(10)

In [ ]:
# Fit all temperature curves with above function
# savgol_filter used to filter a little bit
# smooth the temperatures that oscillates a bit too much

T_max = 85
T_thr = 1
fit_param = zeros((imax,5))
fit_pcov  = zeros((imax,5,5))
my_order = []
gone_wrong = []

figure(1); clf()
for i in range(0,imax):
    print('<<<',i,'>>>')
    k_max = argmin(abs(Temperature[i] - T_max))
    k_thr = argmin(abs(Temperature[i] - T_thr))

    i_aux = where(diff(abs(diff(Temperature[i][k_thr:k_max])))>2)
    if size(i_aux) != 0 :
        if (i_aux[0][0]+k_thr < k_max):
            k_max = i_aux[0][0]+k_thr

    k_min = k_thr - int(2*(k_max - k_thr))
    if k_min < 0: k_min = 0
#         print(k_min)

    x  = Time[i][k_min:k_max]
    print(Urf[i])
    print(len(Temperature[i]),k_max,k_thr,k_min)
    try:
        y  = savgol_filter(Temperature[i][k_min:k_max],31,1)
    except:
        y  = Temperature[i][k_min:k_max]
        print('no filter')
        
    x0 = Time[i][k_thr]

    A0  = 1
    B0  = log10(Temperature[i][k_thr])
    C0  = 1.0e-8
    D0  = 0.5
    p0  = array([x0, A0, B0,  C0, D0])
    
    my_order.append(i)

    try:
        popt, pcov = curve_fit(func2, x, log10(y), p0, maxfev= 500000)
        fit_param[i,:] = popt[0], popt[1], popt[2], popt[3], popt[4]
        fit_pcov[i,:]  = pcov

        y_fit = func2(x,popt[0], popt[1], popt[2], popt[3], popt[4])
        y_fit = 10**y_fit

        subplot(6,4,i+1)
        plot(x*1e3, y    ,label=' Vrf = '+str(Urf[i]))
        plot(x*1e3, y_fit,label='Fit',ls='--')
        grid()
        legend(fontsize=10)
        # select cases
        # non zero fit value
    except TypeError:
        print('TypeError for ',Urf[i])
        gone_wrong.append(i)
tight_layout()

In [ ]:
# select only Urf values
# for which non zero fit
# and sort ascending Urf
Y = Urf
X = my_order
my_order = [x for _,x in sorted(zip(Y,X))]
Urf_order = [x for x,_ in sorted(zip(Y,X))]
# delete manually
# other innapropriate values
my_order.remove(17)
Urf_order.remove(75)

print(Urf_order)
print(my_order)

# resort all variables
# in order so
# Urf ascending
imax_order = len(Urf_order)
fit_param_order = zeros((imax_order,5))
fit_pcov_order  = zeros((imax_order,5,5))
Time_order = []
Temperature_order = []
for i,j in enumerate(my_order):
    fit_param_order[i,:] = fit_param[j,:]
    fit_pcov_order[i,:]  = fit_pcov[j,:]
    Time_order.append(Time[j])
    Temperature_order.append(Temperature[j])

In [ ]:
# draw one temp and fit
figure(3); clf()
cm = pylab.get_cmap('jet')

i = 0

print(Urf_order[i])
# print(fit_pcov_order[i,:])
y_fit = func2(Time_order[i],
              fit_param_order[i,0],
              fit_param_order[i,1],
              fit_param_order[i,2],
              fit_param_order[i,3],
              fit_param_order[i,4])
y_fit = 10**y_fit
subplot(121)
plot((Time_order[i])*1e3,Temperature_order[i],color='r')
plot((Time_order[i])*1e3,y_fit,'--',color='k')
xlabel('time [ms]')
ylabel('T [K]')
grid()

# xlim(3.6,3.7)
# ylim(-1,10)
subplot(122)
semilogy((Time_order[i])*1e3,savgol_filter(Temperature_order[i],31,1),color='r')
semilogy((Time_order[i])*1e3,y_fit,'--',color='k')
xlabel('time [ms]')
grid()
tight_layout()

In [ ]:
# draw fit parameters
# as a function of Urf
str_labels = ['t0', 'A', 'B', 'C', 'D']
figure('Param'); clf();
for i in range(0,5):
    subplot(1,5,i+1)
    if i == 2:
        plot(Urf_order,10**(fit_param_order[:,i]),'o-', label=str_labels[i])
    else:
        plot(Urf_order,(fit_param_order[:,i]),'o-', label=str_labels[i])
    xlabel('Urf')
    grid()
    legend()

In [ ]:
# Draw Heating rate H
# as a function of temperature T
xa = -2
xb = 50
ya = 6e-6
yb = 1e0
figure(29,clear='True')
ax = subplot(111)
for i in [0,1,13,17]: #  range(0,imax_order)
    t0, A, B, C, D = fit_param_order[i,:]
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D)
#     ax.semilogy((Time_order[i]-t0)*1e3,dy*1e-6,label=str(Urf[i]),color=cm(i/imax_order))
    ax.semilogy(savgol_filter(Temperature_order[i],31,1),dy*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order))
#     ax.semilogy(Temperature_order[i][:-1],diff(Temperature_order[i])*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order))
    ax.set_xlabel('T [K]')
    ax.set_ylabel('Heating rate [K/µs]')
#     scatter(t0,4e5,color=cm(i/imax),marker='+')
#     semilogy(y,dy*1e-3,label=str(Urf[i]),color=cm(i/imax))
ax.set_xlim([xa,xb])
ax.set_ylim([ya,yb])
        
# locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
# ax.yaxis.set_major_locator(locmaj)
# locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
# ax.yaxis.set_minor_locator(locmin)
# ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax.legend(title=r'$U_{RF}$',ncol=2)
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
tight_layout()

In [ ]:
# Draw Heating rate H
# as a function of temperature T
xa = -2
xb = 50
ya = 6e-6
yb = 1e0
coucou = []
figure(31,clear='True')
ax = subplot(111)
for i in [0,1,13,17]: #  range(0,imax_order)
    t0, A, B, C, D = fit_param_order[i,:]
    x  = savgol_filter(Temperature_order[i],31,1)
    y  = func2(Time_order[i],t0, A, B, C, D);
    y  = 10**y
    dy = dfunc2_dt(Time_order[i], t0, A, B,  C, D) /x
#     ax.semilogy((Time_order[i]-t0)*1e3,dy*1e-6,label=str(Urf[i]),color=cm(i/imax_order))
    ax.semilogy(x,dy*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order),marker='+')
#     ax.semilogy(Temperature_order[i][:-1],diff(Temperature_order[i])*1e-6,label=str(Urf_order[i]),color=cm(i/imax_order))
    coucou.append( argmin(abs(x-0.5)) )

#     ax.vlines(x[coucou],1e-5,0.5e0,color=cm(i/imax_order))
    ax.set_xlabel('T [K]')
    ax.set_ylabel('Heating rate / T [/µs]')
#     scatter(t0,4e5,color=cm(i/imax),marker='+')
#     semilogy(y,dy*1e-3,label=str(Urf[i]),color=cm(i/imax))
ax.set_xlim([xa,xb])
ax.set_ylim([ya,yb])
        
# locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
# ax.yaxis.set_major_locator(locmaj)
# locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
# ax.yaxis.set_minor_locator(locmin)
# ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax.legend(title=r'$U_{RF}$',ncol=2)
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')
tight_layout()

In [ ]:
X = []
Y = []
for i,j in enumerate([0,1,13,17]):
#     x = savgol_filter(Temperature_order[i],31,1)
    x  = func2(Time_order[i],t0, A, B, C, D)
    X.append(Urf_order[j])
    Y.append(x[coucou[i]])
print(X,Y)
figure('test',clear='True')
plot(X,Y)

In [ ]:
110/3493 * 15, x[15]

# t0 and T as a function of Urf

In [ ]:
t0    = zeros(imax_order)
y_t0  = zeros(imax_order)
dy_t0 = zeros(imax_order)
for i in range(0, imax_order):
    t0[i], A, B, C, D = fit_param_order[i,:]
    y  = func2(t0[i],t0[i], A, B, C, D);
    y_t0[i]  = 10**y
    dy_t0[i] = dfunc2_dt(t0[i], t0[i], A, B,  C, D)

In [ ]:
def my_lin(x,a,b):
    return a*x+b
def my_power_law(x,a,b):
    return a*(x**-b)

In [ ]:
# log plot
figure(321,clear='True')
ax = subplot(111)
ax.loglog(Urf_order,t0*1e3,ls='-',marker='+',mew=5,ms=10)
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$t_0$ [ms]') # '$T(t_0) / t_0$ [K/s]'
ax.axis([5*10, 72, 1, 200])
ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')

tight_layout()

In [ ]:
figure(322); clf();
ax = subplot(111)
ax.plot(Urf_order,y_t0,ls='-',marker='+',mew=5,ms=10)
ax.grid()
ax.set_xlabel('$U_{rf}$ [V]')
ax.set_ylabel('$T(t_0)$ [K]') # '$T(t_0) / t_0$ [K/s]'

popt, pcov = curve_fit(my_lin,Urf_order[2:-2],y_t0[2:-2])
ax.plot(Urf_order[2:-2],my_lin(array(Urf_order[2:-2]),popt[0],popt[1]),
    color='k',ls='--')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=5))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))

ax.text(64,0.815,f'$T(t_0) = {popt[0]:.2e} U_{{rf}} + {popt[1]:.2f}$',
            color='xkcd:black',fontsize='20',weight="bold",
            ha='left',va='center')

tight_layout()

# identique à 10**B

In [ ]:
figure(323); clf();
subplot(111)
plot(Urf_order,y_t0,ls='-',marker='+',mew=5,ms=10)
grid()
xlabel('$U_{rf}$ [V]')
ylabel('$T(t_0)$ [K]') # '$T(t_0) / t_0$ [K/s]'

tight_layout()

# identique à 10**B

In [ ]:
# check for frequencies